In [80]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

In [81]:
df = pd.read_csv('C:\\Users\Asus\PycharmProjects\\bigDataLabs\seventh_lab\winemag-data_first150k.csv', encoding='latin-1')
df.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum SelecciÃ³n Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen RodrÃ­guez
2,2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,4,France,"This is the top wine from La BÃ©gude, named af...",La BrÃ»lade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la BÃ©gude


In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150930 entries, 0 to 150929
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Unnamed: 0   150930 non-null  int64  
 1   country      150925 non-null  object 
 2   description  150930 non-null  object 
 3   designation  105195 non-null  object 
 4   points       150930 non-null  int64  
 5   price        137235 non-null  float64
 6   province     150925 non-null  object 
 7   region_1     125870 non-null  object 
 8   region_2     60953 non-null   object 
 9   variety      150930 non-null  object 
 10  winery       150930 non-null  object 
dtypes: float64(1), int64(2), object(8)
memory usage: 12.7+ MB


In [83]:
df.keys()

Index(['Unnamed: 0', 'country', 'description', 'designation', 'points',
       'price', 'province', 'region_1', 'region_2', 'variety', 'winery'],
      dtype='object')

In [84]:
df['target'] = df['points'].apply(lambda x: 'neg' if x <= 85 else 'pos')

In [85]:
df['target']

0         pos
1         pos
2         pos
3         pos
4         pos
         ... 
150925    pos
150926    pos
150927    pos
150928    pos
150929    pos
Name: target, Length: 150930, dtype: object

In [86]:
review_counts = df['target'].value_counts()
review_counts

target
pos    115322
neg     35608
Name: count, dtype: int64

In [87]:
df.drop(columns=['price'], inplace=True)
df.drop(columns=['province'], inplace=True)
df.drop(columns=['region_1'], inplace=True)
df.drop(columns=['region_2'], inplace=True)

In [88]:
df.keys()

Index(['Unnamed: 0', 'country', 'description', 'designation', 'points',
       'variety', 'winery', 'target'],
      dtype='object')

In [89]:
text_train, text_test, y_train, y_test = train_test_split(df['description'], df['target'], test_size=0.2, random_state=42)

In [90]:
vect = CountVectorizer().fit(text_train)
X_train = vect.transform(text_train)
print('X_train:\n{}'.format(repr(X_train)))

X_train:
<120744x30587 sparse matrix of type '<class 'numpy.int64'>'
	with 4139402 stored elements in Compressed Sparse Row format>


In [91]:
feature_names = vect.get_feature_names_out()
print('Количество признаков: {}'.format(len(feature_names)))
print('Первые 20 признаков: {}'.format(feature_names[:20]))
print('Признаки с 1000 до 1020: {}'.format(feature_names[1000:1020]))

Количество признаков: 30587
Первые 20 признаков: ['00' '000' '002' '008' '01' '01s' '02' '02s' '03' '03s' '04' '04s' '04â'
 '05' '05s' '06' '061' '064' '06s' '07']
Признаки с 1000 до 1020: ['aforementioned' 'afoul' 'afraid' 'afresh' 'africa' 'african' 'africans'
 'afrikaner' 'aft' 'after' 'afterburners' 'afterflavors' 'afterglow'
 'afterlife' 'aftermarket' 'aftermath' 'afternoon' 'afternoons'
 'afternoonâ' 'afternotes']


In [92]:
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

# Check class distribution
unique_classes, class_counts = np.unique(y_train, return_counts=True)
print("Class Distribution:", dict(zip(unique_classes, class_counts)))

# Continue with cross-validation if classes are balanced
if len(unique_classes) >= 2:
    scores = cross_val_score(LogisticRegression(max_iter=1000), X_train, y_train, cv=5)
    print("Average cross-validation accuracy: {:.2f}".format(np.mean(scores)))
else:
    print("Error: Not enough classes in the training data.")


Class Distribution: {'neg': 28505, 'pos': 92239}
Average cross-validation accuracy: 0.90


In [93]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# Check class distribution
unique_classes, class_counts = np.unique(y_train, return_counts=True)
print("Class Distribution:", dict(zip(unique_classes, class_counts)))

# Continue with grid search if classes are balanced
if len(unique_classes) >= 2:
    param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}
    grid = GridSearchCV(LogisticRegression(max_iter=3000), param_grid, cv=5)
    grid.fit(X_train, y_train)
    print('Best Cross-Validation Score: {:.2f}'.format(grid.best_score_))
    print('Best Parameters: ', grid.best_params_)
else:
    print("Error: Not enough classes in the training data.")


Class Distribution: {'neg': 28505, 'pos': 92239}
Best Cross-Validation Score: 0.90
Best Parameters:  {'C': 10}


In [94]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(min_df=10).fit(text_train)
X_train = vect.transform(text_train)
print("X_train c min_df: {}".format(repr(X_train)))

X_train c min_df: <120744x9232 sparse matrix of type '<class 'numpy.int64'>'
	with 4080223 stored elements in Compressed Sparse Row format>


In [96]:
feature_names = vect.get_feature_names_out()
print('Количество признаков: {}'.format(len(feature_names)))
print('Первые 20 признаков: {}'.format(feature_names[:20]))
print('Признаки с 1000 до 1020: {}'.format(feature_names[1000:1020]))

Количество признаков: 9232
Первые 20 признаков: ['000' '01' '02' '03' '04' '05' '06' '07' '08' '09' '10' '100' '10â' '11'
 '12' '125' '13' '130' '14' '15']
Признаки с 1000 до 1020: ['bolla' 'bollinger' 'bolster' 'bolstered' 'bolstering' 'bolt' 'bomb'
 'bombs' 'bon' 'bonarda' 'bone' 'boned' 'bones' 'bonterra' 'bonus' 'book'
 'bookwalter' 'boost' 'boosted' 'boosts']


In [99]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Получение списка текстовых отзывов из столбца 'Review'
reviews = df['description'].tolist()
nltk.download('stopwords')
nltk.download('punkt')

# Создание словаря стоп-слов на основе английского языка из библиотеки nltk
stop_words = set(stopwords.words('english'))

# Пример токенизации текста
tokenized_reviews = [word_tokenize(review) for review in reviews]

# Удаление стоп-слов из токенизированных отзывов
filtered_reviews = [[word for word in review if word.lower() not in stop_words] for review in tokenized_reviews]

# Пример использования CountVectorizer с удалением стоп-слов
vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(reviews)

# Получение списка всех слов (без учета стоп-слов) из CountVectorizer
feature_names = vectorizer.get_feature_names_out()

# Вывод примера результата
print("Original Review:", reviews[0])
print("Tokenized Review:", tokenized_reviews[0])
print("Filtered Review (without stopwords):", filtered_reviews[0])
print("Features (words) from CountVectorizer:", feature_names)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Original Review: This tremendous 100% varietal wine hails from Oakville and was aged over three years in oak. Juicy red-cherry fruit and a compelling hint of caramel greet the palate, framed by elegant, fine tannins and a subtle minty tone in the background. Balanced and rewarding from start to finish, it has years ahead of it to develop further nuance. Enjoy 2022â2030.
Tokenized Review: ['This', 'tremendous', '100', '%', 'varietal', 'wine', 'hails', 'from', 'Oakville', 'and', 'was', 'aged', 'over', 'three', 'years', 'in', 'oak', '.', 'Juicy', 'red-cherry', 'fruit', 'and', 'a', 'compelling', 'hint', 'of', 'caramel', 'greet', 'the', 'palate', ',', 'framed', 'by', 'elegant', ',', 'fine', 'tannins', 'and', 'a', 'subtle', 'minty', 'tone', 'in', 'the', 'background', '.', 'Balanced', 'and', 'rewarding', 'from', 'start', 'to', 'finish', ',', 'it', 'has', 'years', 'ahead', 'of', 'it', 'to', 'develop', 'further', 'nuance', '.', 'Enjoy', '2022â\x80\x932030', '.']
Filtered Review (without stopw

In [101]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Пример, используя ваш DataFrame df и столбец 'Review'
corpus = df['description'].tolist()

# Создание экземпляра TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.85, min_df=0.1, max_features=500)

# Преобразование текста в TF-IDF матрицу
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

# Получение списка фич (слов) из TfidfVectorizer
feature_names_tfidf = tfidf_vectorizer.get_feature_names_out()

# Создание DataFrame для отображения результата
tfidf_df = pd.DataFrame(data=tfidf_matrix.toarray(), columns=feature_names_tfidf)

# Вывод первых нескольких строк DataFrame с TF-IDF данными
print(tfidf_df.head())


    acidity    aromas  berry     black    cherry     drink  dry    finish  \
0  0.000000  0.000000    0.0  0.000000  0.358132  0.000000  0.0  0.332754   
1  0.000000  0.331825    0.0  0.387255  0.000000  0.385041  0.0  0.324013   
2  0.000000  0.000000    0.0  0.000000  0.000000  0.000000  0.0  0.000000   
3  0.000000  0.323290    0.0  0.377295  0.339755  0.375138  0.0  0.315680   
4  0.401295  0.000000    0.0  0.000000  0.000000  0.447122  0.0  0.000000   

    flavors  fresh  ...    palate       red      rich      ripe  shows  soft  \
0  0.000000    0.0  ...  0.369114  0.428951  0.000000  0.000000    0.0   0.0   
1  0.230886    0.0  ...  0.359418  0.000000  0.404831  0.374575    0.0   0.0   
2  0.709279    0.0  ...  0.000000  0.000000  0.000000  0.000000    0.0   0.0   
3  0.224948    0.0  ...  0.000000  0.000000  0.000000  0.000000    0.0   0.0   
4  0.000000    0.0  ...  0.000000  0.000000  0.000000  0.000000    0.0   0.0   

   spice  sweet   tannins      wine  
0    0.0    0.0  0

In [104]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

corpus = df['description'].tolist()

# Создание экземпляра TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.85, min_df=0.1, max_features=500)

# Преобразование текста в TF-IDF матрицу
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, df['points'], test_size=0.2, random_state=42)

# Создание и обучение модели (в примере - линейная регрессия)
model = LinearRegression()
model.fit(X_train, y_train)

# Исследование коэффициентов модели
coefficients = pd.DataFrame({'Feature': tfidf_vectorizer.get_feature_names_out(), 'Coefficient': model.coef_})
coefficients = coefficients.sort_values(by='Coefficient', ascending=False)

# Вывод топ-10 положительных и топ-10 отрицательных коэффициентов
print("Top 10 Positive Coefficients:")
print(coefficients.head(10))

print("\nTop 10 Negative Coefficients:")
print(coefficients.tail(10))


Top 10 Positive Coefficients:
    Feature  Coefficient
17     rich     3.611360
3     black     2.731043
21    spice     2.250189
23  tannins     2.113675
24     wine     2.046003
18     ripe     2.038080
19    shows     1.750969
5     drink     1.541988
14      oak     1.389059
10    fruit     1.186422

Top 10 Negative Coefficients:
    Feature  Coefficient
4    cherry     0.126278
16      red     0.124688
2     berry     0.106949
6       dry    -0.083238
1    aromas    -0.271765
11     good    -0.291180
12     nose    -0.599124
22    sweet    -0.765701
8   flavors    -0.861996
20     soft    -1.230367


<h1>Выводы</h1>

<h2>1. Подготовка данных:</h2>
   - Я загрузила данные из CSV-файла и провели предварительный анализ, включая информацию о данных и создание бинарной целевой переменной 'target' на основе баллов.

<h2>2. Текстовые признаки:</h2>
   - Я использовала CountVectorizer и TfidfVectorizer для преобразования текстовых отзывов в числовые признаки, которые можно использовать для обучения модели.
   - Произведено исследование признаков, включая вывод списка слов и анализ их распределения.

<h2>3. Классификация с использованием логистической регрессии:</h2>
   - Я использовали логистическую регрессию для классификации отзывов на положительные ('pos') и отрицательные ('neg').
   - Провела кросс-валидацию и оптимизацию параметра регуляризации с использованием GridSearchCV.

<h2>4. TF-IDF и регрессия:</h2>
   - Я применила TF-IDF преобразование текста и обучили линейную регрессию для предсказания баллов.
   - Исследовала коэффициенты модели, что позволяет определить, какие слова сильнее всего влияют на предсказание баллов.

<h2>Выводы:</h2>5. 
   - Применение методов векторизации текста (CountVectorizer, TfidfVectorizer) и машинного обучения позволило мне извлекать информацию из текстовых данных.
   - Оптимизация параметров модели и исследование коэффициентов помогло понять вклад каждого признака в предсказание.
